In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Create a set of random latitudes and longitude combinations
lats = np.random.uniform(-90.000,90.000,1500)
longs = np.random.uniform(-180.000,180.000,1500)

In [ ]:
# Combine both arrays
lats_longs = zip(lats,longs)
lats_longs


In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_longs)


In [ ]:
# Use the tuple() function to display the latitude and longitude combinations.
# city_name and country_code can be used to get the cities and their country code
# for cord in coordinates:
#     print(citipy.nearest_city(cord[0],cord[1]).city_name,
#           citipy.nearest_city(cord[0],cord[1]).country_code)

In [ ]:
# Creating a list for holding cities
cities = []
for cord in coordinates:
    city = citipy.nearest_city(cord[0],cord[1]).city_name

    # Check for duplicates
    if city not in cities:
        cities.append(city)

# Get length of city list
len(cities)

In [ ]:
# Import requests library
import requests

# Import Weather API Key
from config import weather_api_key

# Import datetime
from datetime import datetime

In [ ]:
# Starting URL for Weather Map API Call.
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=' + weather_api_key
print(url)

In [ ]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

In [ ]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather

In [ ]:
# Get JSON text of the Get request
city_weather.json()

In [ ]:
# Create an endpoint URL for a city
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
if city_weather.status_code == 200:
    print(f'City Weather found')
else:
    print(f'City Weather not found.')

In [ ]:
# Get the JSON
boston_data = city_weather.json()
country = boston_data['sys']['country']
city = boston_data['name']
lat = boston_data['coord']['lat']
lng = boston_data['coord']['lon']
max_temp = boston_data['main']['temp_max']
humidity = boston_data['main']['humidity']
clouds = boston_data['clouds']['all']
wind = boston_data['wind']['speed']

In [ ]:
print(country, city, lat, lng, max_temp, humidity, clouds, wind)

In [ ]:
# Get the date from the weather
date = boston_data['dt']

# convert UTC date to date format year, month, day, hour, minute, second
# date = datetime.utcfromtimestamp(date)

date = datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

# date

In [ ]:
# Create an empty ;list to hold the data
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counter
record_count = 1
set_count = 1

# loop through all the cities
for i, city in enumerate(cities):

    # Grouping cities in sets of 50 for logging
    if (i % 50 == 0 and i >= 50 ):
        set_count += 1
        record_count = 1

    city_url = url + "&q=" + city.replace(" ", "+")

    # Logging the URL, record number, set number and city
    print(f'Processing record {record_count} of Set {set_count} | {city}')
    record_count += 1

    try:
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')

        city_data.append({'City': city.title(),
            'Latitude': city_lat,
            'Longitude': city_lng,
            'Humidity': city_humidity,
            'Cloudiness': city_clouds,
            'Humidity': city_max_temp,
            'Winds': city_wind,
            'Country': city_country,
            'Date': city_date
        })
    except:
        print(f'The City {city} not Found. Skipping .....')

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
city_data[0]

In [ ]:
city_data_df = pd.DataFrame(city_data)
temp = city_data_df
city_data_df.head()

In [ ]:
# Re-order the columns
new_order = ['City', 'Country', 'Date', 'Latitude', 'Longitude',
             'Humidity', 'Humidity', 'Cloudiness','Winds']

In [ ]:
city_data_df = city_data_df[new_order]
city_data_df

In [ ]:
# Create an output file
output_file = 'weather_data/cities.csv'

# Export to csv
city_data_df.to_csv(output_file,index_label='City_ID')


### Plotting the Latitude vs Temperature


In [ ]:
# Getting Data for plotting
lat = city_data_df['Latitude']
lng = city_data_df['Longitude']
max_temp = city_data_df['Humidity']
humidity = city_data_df['Humidity']
cloudiness = city_data_df['Cloudiness']
wind_speed = city_data_df['Winds']

In [ ]:
# Import the time module
import time

today = time.time()
today

In [ ]:
# Building the scatter plot
plt.title(f'City Latitude vs Max. Temperature (F) {time.strftime("%x")}')
plt.scatter(lat, max_temp,
            edgecolors='black', alpha=0.8, marker='o',
            linewidths=1, label='Cities')
plt.xlabel('Latitude')
plt.ylabel('Humidity (F)')
plt.grid(True)

# Save the figure
plt.savefig('weather_data/Fig1.png')

# Show plot
plt.show()


### Plotting the Latitude vs Humidity

In [ ]:
# Building the scatter plot
plt.title(f'City Latitude vs Humidity (%) {time.strftime("%x")}')
plt.scatter(lat, humidity,
            edgecolors='black', alpha=0.8, marker='o',
            linewidths=1, label='Cities')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid(True)

# Save the figure
plt.savefig('weather_data/Fig2.png')

# Show plot
plt.show()



### Plotting the Latitude vs Cloudiness

In [ ]:
plt.subplot()
plt.title(f'City Latitude vs Cloudiness (%) {time.strftime("%x")}')
plt.scatter(lat, cloudiness,
            edgecolors='black', alpha=0.8, marker='o',
            linewidths=1, label='Cities')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.grid(True)

# Save the figure
plt.savefig('weather_data/Fig3.png')

# Show plot
plt.show()


In [ ]:
# Importing the stats module
from scipy.stats import linregress

### Plotting the Latitude vs Cloudiness

In [ ]:
plt.title(f'City Latitude vs Wind Speed(%) {time.strftime("%x")}')
plt.scatter(lat, wind_speed,
            edgecolors='black', alpha=0.8, marker='o',
            linewidths=1, label='Cities')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.grid(True)

# Save the figure
plt.savefig('weather_data/Fig4.png')

# Show plot
plt.show()


## Custom Functions for Regression and the Plots.

In [ ]:
# Function to calculate the linear regression and and get the equation of the line.
# Also get the the independent variable for each dependent variable
def linear_regression_values(x,y):
    (slope, intercept, r_value, p_value, std_err) = linregress(x, y)
    temp_eq_line = f'y = {round(slope, 2)}(x) + {round(intercept,2)}'
    regression_list = [i*slope + intercept for i in x]
    return r_value

def linear_regression_plot(x,y, plot_title, y_label, text_coordinates):

    # Run regression on the data
    (slope, intercept, r_value, p_value, std_err) = linregress(x, y)
    regression_list = [i*slope + intercept for i in x]
    temp_eq_line = f'y = {round(slope, 2)}(x) + {round(intercept,2)}'

    # Create the scatter plot and the Regression line plot for the data
    plt.title(plot_title, fontsize=15)
    plt.scatter(x,y, edgecolors='black')
    plt.plot(x,regression_list, color='red')

    # Labels for plot
    plt.xlabel('Latitude', fontsize=12)
    plt.ylabel(y_label, fontsize=12)

    # Annotate the graph by putting the equation of line
    plt.annotate(temp_eq_line,text_coordinates, fontsize=15, color='r')
    plt.show()

## Correlation between Latitude and Temperature.

In [ ]:
city_data_df.head()

In [ ]:
# Getting the northern and southern hemisphere
north_hemisphere = city_data_df.loc[city_data_df['Latitude'] >= 0]
south_hemisphere = city_data_df.loc[city_data_df['Latitude'] < 0]

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(north_hemisphere['Latitude'], north_hemisphere['Humidity'],
                       'Linear Regression on the Northern Hemisphere for Maximum Temperature',
                       text_coordinates=(10,0), y_label='Maximum Temperature')

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(south_hemisphere['Latitude'], south_hemisphere['Humidity'],
                       'Linear Regression on the Southern Hemisphere for Maximum Temperature',
                       text_coordinates=(-55,100), y_label='Maximum Temperature')


In [ ]:
# R values of the regression
north_r_value = linear_regression_values(north_hemisphere['Latitude'], north_hemisphere['Humidity'])
south_r_value = linear_regression_values(south_hemisphere['Latitude'], south_hemisphere['Humidity'])
north_r_value,south_r_value

## Correlation between Latitude and Percent Humidity.

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(north_hemisphere['Latitude'], north_hemisphere['Humidity'],
                       'Linear Regression on the Northern Hemisphere for % Humidity',
                       text_coordinates=(40,0), y_label='% Humidity')

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(south_hemisphere['Latitude'], south_hemisphere['Humidity'],
                       'Linear Regression on the Southern Hemisphere for % Humidity',
                       text_coordinates=(-55,10), y_label='% Humidity')

In [ ]:
# R values of the regression
north_humidity_r_value = linear_regression_values(north_hemisphere['Latitude'], north_hemisphere['Humidity'])
south_humidity_r_value = linear_regression_values(south_hemisphere['Latitude'], south_hemisphere['Humidity'])
north_humidity_r_value,south_humidity_r_value

## Correlation between Latitude and Percent Cloudiness.

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(north_hemisphere['Latitude'], north_hemisphere['Cloudiness'],
                       'Linear Regression on the Northern Hemisphere for % Cloudiness',
                       text_coordinates=(10,32), y_label='% Cloudiness')

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(south_hemisphere['Latitude'], south_hemisphere['Cloudiness'],
                       'Linear Regression on the Southern Hemisphere for % Cloudiness',
                       text_coordinates=(-55,17), y_label='% Cloudiness')

In [ ]:
# R values of the regression
north_Cloudiness_r_value = linear_regression_values(north_hemisphere['Latitude'], north_hemisphere['Cloudiness'])
south_Cloudiness_r_value = linear_regression_values(south_hemisphere['Latitude'], south_hemisphere['Cloudiness'])
north_Cloudiness_r_value,south_Cloudiness_r_value

## Correlation between Latitude and Percent Winds.

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(north_hemisphere['Latitude'], north_hemisphere['Winds'],
                       'Linear Regression on the Northern Hemisphere for Wind Speed',
                       text_coordinates=(10,40), y_label='Wind Speed (mph)')

In [ ]:
# Linear regression on the Northern Hemisphere
linear_regression_plot(south_hemisphere['Latitude'], south_hemisphere['Winds'],
                       'Linear Regression on the Southern Hemisphere for Wind Speed',
                       text_coordinates=(-55,17), y_label='Wind Speed (mph)')

In [ ]:
# R values of the regression
north_Winds_r_value = linear_regression_values(north_hemisphere['Latitude'], north_hemisphere['Winds'])
south_Winds_r_value = linear_regression_values(south_hemisphere['Latitude'], south_hemisphere['Winds'])
north_Winds_r_value,south_Winds_r_value


In [ ]:
from test import api_key

In [ ]:
api_key


In [ ]:

#%